In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [2]:
train_data = pd.read_csv('train.csv/train.csv')
test_data = pd.read_csv('train.csv/test.csv')

In [3]:
# train_data.drop(['cont13'], axis = 1, inplace = True)
# test_data.drop(['cont13'], axis = 1, inplace = True)

In [4]:
cat_cols = [col for col in train_data if train_data[col].dtype == 'object']
print(cat_cols)
num_cols = [col for col in train_data if train_data[col].dtype == 'float64']
print(num_cols)
feat_cols = [col for col in train_data.columns if col != 'target']
# feat_cols.pop(0)
print(feat_cols)

['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'target']
['id', 'cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']


In [5]:
y = train_data['target']
X = train_data[feat_cols]

final_X = test_data[feat_cols]

In [6]:
from sklearn.preprocessing import OrdinalEncoder

In [7]:
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state = 3, test_size = 0.15)

In [8]:
label_train_x = train_X.copy()
label_test_x = test_X.copy()

ordinal_encoder = OrdinalEncoder()

label_train_x[cat_cols] = ordinal_encoder.fit_transform(train_X[cat_cols])
label_test_x[cat_cols] = ordinal_encoder.transform(test_X[cat_cols])

In [9]:
xgb = XGBRegressor(n_estimators = 3850, max_depth = 2, learning_rate = 0.1)
xgb.fit(label_train_x, train_y)
score = xgb.predict(label_test_x)
score = mean_squared_error(test_y, score, squared=False)

In [10]:
print(f'root mean squared error value: {score:.5f}')

root mean squared error value: 0.71767


In [11]:
new_data_test = test_data.copy()
new_data_test[cat_cols] = ordinal_encoder.fit_transform(test_data[cat_cols])

In [12]:
# new_data_test.drop(['id'], axis = 1, inplace=True)

In [13]:
prediction_data = new_data_test
predict = xgb.predict(prediction_data)

In [14]:
output = pd.DataFrame({'id' : test_data['id'],
                      'target' : predict
                      })

In [15]:
output.to_csv('submission.csv', index=False)